# Fitting First Order ODE Solutions to Given Data


## Solving First Order ODE

Let $y(t)$ be defined by a system of first order ODEs which involve some unknown parameters $p = (p_1, \ldots, p_l)$. That is

$$\frac{dy(t)}{dt} = f(y(t), t, p)$$

Here, $y$ can be an $\mathbb{R}$-valued or vector-valued function. If $y(t) = (y_1(t), \ldots, y_k(t))$, then $\frac{dy(t)}{dt} = (\frac{dy_1(t)}{dt}, \ldots, \frac{dy_k(t)}{dt})$.

### Task 1. Simple harmonic oscillation

The data in the file `simple-harmonic-noisy.csv` is a noisy reading of a function $y(t)$ defined by the following second order ODE
$$\frac{d^2}{dt^2} y + \omega^2 y = 0$$
with initial conditions $y(0) = 0$ and $y_1(0) = 0$ in the interval $[0,10]$.

Estimate the value of $\omega$ using
1. scipy.optimize.minimize (try multiple methods and time the successfull ones)
2. scipy.optimize.least_squares (try multiple methods and time the successfull ones)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as integrate
import scipy.optimize as optimize
import pandas as pd
import time

In [2]:
# Read the data from csv file into a pandas dataframe
df = pd.read_csv("/Users/adityamishra/Documents/Machine Learning Tutorial/1. Machine Learning Specialisation/Important-ML-Implementations/10-Curve-Fitting/data/simple-harmonic-noisy.csv")
print(df.head())

      time         y
0  0.00000 -0.046736
1  0.10101  0.231668
2  0.20202  0.471371
3  0.30303  0.705724
4  0.40404  0.835048


In [3]:
# Storing the data in the numpy arrays
t_eval = np.array(df["time"])# definig t_eval as the time values array
y_real = np.array(df["y"])

In [4]:
# Setting up the IVP: ODE, initial conditions
def harmonic_ode(t,y,omega):
    y_current, y1_current = y # unpacking the y array into y_current and y1_current
    dy_dt = y1_current
    dy1_dt = -(omega ** 2) * y_current
    return [dy_dt, dy1_dt]

#Definding the initial conditions
y0 = [0,1]

In [9]:
# Define residual and cost (least square error) between real data and estimate
def residual(p):
   sol = integrate.solve_ivp(harmonic_ode,(t_eval[0], t_eval[-1]),y0,t_eval=t_eval,args=tuple(p), method='RK45')
   return sol.y[0] - y_real

def cost(p):
    return np.sum(residual(p) ** 2)

In [10]:
# Find omega using scipy.optimize.minimize
p_guess = [3.0]
methods_min = ['Nelder-Mead', 'Powell']

print("Estimating the value of omega using scipy.optimize.minimize: ")
results_min = []# list to store the results of the optimization
for method in methods_min:
    start_time = time.time()# start time of the optimization
    result = optimize.minimize(cost, p_guess, method=method)
    elapsed_time = time.time() - start_time# time taken to optimize
    if result.success:
        print(f"Method:  {method} -> ω = {result.x[0]:.4f}, Cost: {result.fun:.2f}, Time: {elapsed_time:.2f}s")# printing the results where method is the optimization method, result.x[0] is the optimized value of omega, result.fun is the cost and elapsed_time is the time taken to optimize
    else:
        print(f"Method:  {method} Failed.")
         

Estimating the value of omega using scipy.optimize.minimize: 
Method:  Nelder-Mead -> ω = 3.2716, Cost: 51.19, Time: 0.11s
Method:  Powell -> ω = 2.4891, Cost: 17.99, Time: 0.06s


In [ ]:
# Find omega using scipy.optimize.least_squares